# D04 Thay thế dữ liệu: `replace()`, `map()`, `mask()`, và `where()`

## Mục đích

Trong bài này, chúng ta sẽ tìm hiểu những cách thức khác nhau để thay thế dữ liệu trong data frame. Chúng ta sẽ chỉ tiếp cận với các kĩ thuật thay thế dữ liệu cơ bản, các kĩ thuật nâng cao và mở rộng khác bạn có thể xem trong [API Reference](https://pandas.pydata.org/docs/reference/index.html) trên trang của Pandas.


## `replace()`

Cách thức thay thế dữ liệu mà mình thường sử dụng nhất là sử dụng từ điển, và theo hai cách sau.

### Thay thế giá trị bởi giá trị

Bạn có thể thay thế một hoặc nhiều cột có cùng cách mã hóa số liệu. Chẳng hạn trong bộ số liệu HRM, chúng ta có hai cột xét nghiệm *H. pylori* đều mã hóa 0=Âm tính và 1=Dương tính.

In [48]:
import pandas as pd
d = pd.read_excel("hrm.xlsx", index_col="id")
d.filter(like="hp").head(10)

,hp_endo,hp_breath
id,,
223,0.0,NaN
236,0.0,NaN
256,0.0,1.0
296,1.0,99.0
310,0.0,NaN
312,NaN,NaN
326,0.0,1.0
339,1.0,99.0
342,NaN,NaN


Chúng ta có thể thay thế các giá trị 0 và 1 này bằng `"Âm tính"` và `"Dương tính"` để tiện cho việc xuất báo cáo.

In [49]:
d.filter(like="hp").replace({0: "Âm tính", 1: "Dương tính"})

,hp_endo,hp_breath
id,,
223,Âm tính,NaN
236,Âm tính,NaN
256,Âm tính,Dương tính
296,Dương tính,99.0
310,Âm tính,NaN
...,...,...
4200,NaN,NaN
4214,Dương tính,Dương tính
4216,Âm tính,NaN


Chú ý rằng lệnh trên không lưu nội dung thay đổi lại vào bộ nhớ. Nếu muốn thay đổi lại nội dung, bạn cần sử dụng lệnh gán như chúng ta đã học trong bài [D03](./03_slicing.ipynb).

In [50]:
# Dùng lệnh này để lấy danh sách tên cột
cols_hp = list(d.filter(like="hp").columns)

# Thay thế dữ liệu và gán lại vào cột cũ
d.loc[:, cols_hp] = d[cols_hp].replace({0: "Âm tính", 1: "Dương tính"})

d[cols_hp]

,hp_endo,hp_breath
id,,
223,Âm tính,NaN
236,Âm tính,NaN
256,Âm tính,Dương tính
296,Dương tính,99.0
310,Âm tính,NaN
...,...,...
4200,NaN,NaN
4214,Dương tính,Dương tính
4216,Âm tính,NaN


Mình có một thói quen tạo từ điển để thay thế như sau. Nếu bạn chưa quen với `zip()` thì quay lại bài [I01](../02_inter/01_zipenum.ipynb) để ôn lại nhé.

In [51]:
negpos_codes = [0, 1]
negpos_values = ["Âm tính", "Dương tính"]
dict(zip(negpos_codes, negpos_values))

{0: 'Âm tính', 1: 'Dương tính'}

### Thay thế cho một cột cụ thể

Đôi khi bạn muốn thay thế cụ thể cho một cột nào đó (để không ảnh hưởng đến giá trị của những cột khác). Trong trường hợp này, chúng ta sẽ dùng một từ điển lồng (nested dictionary) như sau:

```python
{
    "tên_cột_1": {
        code_1: value_1,
        code_2: value_2,
        ...
    },
    "tên_cột_2": {...}
}
```

Chẳng hạn, nếu muốn thay đổi nội dung của cột giới tính và hai cột xét nghiệm *H. pylori*, mình sẽ tạo một từ điển như sau.

In [52]:
sex_codes = [0, 1]
sex_values = ["Nữ", "Nam"]
sex_dict = dict(zip(sex_codes, sex_values))

negpos_codes = [0, 1]
negpos_values = ["Âm tính", "Dương tính"]
negpos_dict = dict(zip(negpos_codes, negpos_values))

values_dict = {
    "sex": sex_dict,
    "hp_endo": negpos_dict,
    "hp_breath": negpos_dict
}

values_dict

{'sex': {0: 'Nữ', 1: 'Nam'},
 'hp_endo': {0: 'Âm tính', 1: 'Dương tính'},
 'hp_breath': {0: 'Âm tính', 1: 'Dương tính'}}

Và sử dụng hàm `replace()` như sau.

In [53]:
d = pd.read_excel("hrm.xlsx", index_col="id")
d[values_dict.keys()].head(5)

,sex,hp_endo,hp_breath
id,,,
223,1,0.0,NaN
236,1,0.0,NaN
256,1,0.0,1.0
296,1,1.0,99.0
310,0,0.0,NaN


In [54]:
d.replace(values_dict)[values_dict.keys()].head(5)

,sex,hp_endo,hp_breath
id,,,
223,Nam,Âm tính,NaN
236,Nam,Âm tính,NaN
256,Nam,Âm tính,Dương tính
296,Nam,Dương tính,99.0
310,Nữ,Âm tính,NaN


## `map()`

Tính năng của `map()` rất giống với `replace()` ngoại trừ hai điểm: nó chỉ hoạt động trên series, và những giá trị không có chìa khóa của từ điển sẽ tự động được chuyển về NA.

Trong bộ số liệu của chúng ta, bạn có thể thấy bản ghi số 296 có `hp_breath` là `99`, và `replace()` đã giữ nguyên giá trị này. Kết quả sẽ khác nếu bạn dùng `map()`.

In [55]:
d["hp_breath"].map(negpos_dict).iloc[:5]

id
223           NaN
236           NaN
256    Dương tính
296           NaN
310           NaN
Name: hp_breath, dtype: object

## `mask()` và `where()`

Trong nhiều trường hợp bạn sẽ cần thay đổi một số giá trị nếu chúng thỏa mãn điều kiện nào đó, các giá trị không thỏa mãn sẽ được giữ nguyên. Đó là lúc bạn cần đến hàm `mask()`.

Trong ví dụ dưới đây, chúng ta sẽ tạo ra biến `hp_result` là tổng hợp của hai xét nghiệm `hp_endo` và `hp_breath`. Chúng ta sẽ thực hiện việc thay thế như sau:

* Nếu xét nghiệm *H. pylori* trong nội soi không có (NA) thì thay bằng kết quả xét nghiệm trong hơi thở.
* Nếu xét nghiệm *H. pylori* trong hơi thở là dương tính (=1) thì cho dù kết quả xét nghiệm trong nội soi như thế nào cũng sẽ thay bằng 1.

In [56]:
d["hp_result"] = d["hp_endo"].mask(d["hp_endo"].isna(), d["hp_breath"]).mask(d["hp_breath"].eq(1), d["hp_breath"])
d.filter(like="hp").head(10)

,hp_endo,hp_breath,hp_result
id,,,
223,0.0,NaN,0.0
236,0.0,NaN,0.0
256,0.0,1.0,1.0
296,1.0,99.0,1.0
310,0.0,NaN,0.0
312,NaN,NaN,NaN
326,0.0,1.0,1.0
339,1.0,99.0,1.0
342,NaN,NaN,NaN


Cú pháp của hàm `mask()` như sau: `mask(cond=<điều_kiện>, other=<giá_trị_thay_thế_nếu_ĐÚNG>)`.

Ngược lại, nếu bạn muốn thay thế khi điều kiện không thỏa mãn (sai), bạn sẽ dùng hàm `where()`: `where(cond=<điều_kiện>, other=<giá_trị_thay_thế_nếu_SAI>)`. Nếu bạn không thiết lập đối số `other` cho các hàm này, Pandas sẽ mặc định thay thế bằng giá trị NA (`np.nan`).

Chẳng hạn bạn có thể loại bỏ các giá trị không phải là 0 và 1 bằng cách sau (sẽ loại bỏ các giá trị như 99).

In [57]:
d["hp_breath"].where(d["hp_breath"].isin([0, 1])).iloc[:5]

id
223    NaN
236    NaN
256    1.0
296    NaN
310    NaN
Name: hp_breath, dtype: float64

## Luyện tập

Bạn sẽ thấy cột `eso_LA`

In [58]:
d["eso_LA"]

id
223     2.0
236     1.0
256     3.0
296     1.0
310     0.0
       ... 
4200    NaN
4214    0.0
4216    0.0
4220    0.0
4240    0.0
Name: eso_LA, Length: 330, dtype: float64

có một vài giá trị như sau:

In [59]:
d["eso_LA"].unique()

array([ 2.,  1.,  3.,  0., nan])

**Nhiệm vụ 1**: Tạo biến mới tên là `eso_vtq` có giá trị 0 nếu `eso_LA` bằng 0 và 1 với các giá trị còn lại khác NA. Hãy tự suy nghĩ trước khi xem giải thích dưới đây.

Tư duy: Với các biến nhị phân được tạo ra từ biến danh mục có nhiều nhóm, chúng ta có thể sử dụng một điều kiện logic và chuyển giá trị `True`/`False` về 1/0 bằng hàm `astype(int)`.

In [60]:
d["eso_LA"].gt(0).astype(int)

id
223     1
236     1
256     1
296     1
310     0
       ..
4200    0
4214    0
4216    0
4220    0
4240    0
Name: eso_LA, Length: 330, dtype: int32

Tuy nhiên bạn sẽ thấy bản ghi 4200 ban đầu có giá trị NA, nhưng sau khi thay thế dữ liệu lại có giá trị là 0. Lí do là vì theo quy định của NumPy, so sánh `np.nan` với điều kiện `> 0` cho kết quả `False`. Chúng ta cần loại bỏ các giá trị này bằng `mask()`.

In [61]:
d["eso_vtq"] = d["eso_LA"].gt(0).astype(int).mask(d["eso_LA"].isna())
d[["eso_LA", "eso_vtq"]]

,eso_LA,eso_vtq
id,,
223,2.0,1.0
236,1.0,1.0
256,3.0,1.0
296,1.0,1.0
310,0.0,0.0
...,...,...
4200,NaN,NaN
4214,0.0,0.0
4216,0.0,0.0


**Nhiệm vụ 2**: Thay vì tạo giá trị 0 và 1, tạo giá trị là `"Không VTQ"` và `"Có VTQ"` (VTQ là viêm thực quản).

Ở bước này chúng ta có thể dùng `map()` hoặc `replace()` đều được, và sử dụng thay thế cho `astype(int)`.

In [62]:
d["eso_vtq"] = d["eso_LA"].gt(0).map({False: "Không VTQ", True: "Có VTQ"}).mask(d["eso_LA"].isna())
d[["eso_LA", "eso_vtq"]]

,eso_LA,eso_vtq
id,,
223,2.0,Có VTQ
236,1.0,Có VTQ
256,3.0,Có VTQ
296,1.0,Có VTQ
310,0.0,Không VTQ
...,...,...
4200,NaN,NaN
4214,0.0,Không VTQ
4216,0.0,Không VTQ


---

[Bài trước](./03_slicing.ipynb) - [Danh sách bài](../README.md) - [Bài sau]()